In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import os
import dill
import numpy as np
import multiprocessing_on_dill as mp
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sps
import pickle

from collections import defaultdict, Counter
from tqdm import tqdm
from itertools import chain
from cityhash import CityHash64
from itertools import groupby

from f723.tools.urs.extraction import assemble_chains, get_sec_struct_model
from f723.tools.dataset.entities import NucleotideFeatures, PairFeatures, PairMeta, PairData, make_pair, Pair

In [3]:
DATASET_DIR = '/home/mikhail/bioinformatics/data/dataset_all_60'

In [4]:
FEATURES_PATH = os.path.join(DATASET_DIR, 'features.npy')
TARGET_PATH = os.path.join(DATASET_DIR, 'target.npy')
GROUPS_PATH = os.path.join(DATASET_DIR, 'groups.npy')

In [5]:
FEATURES_SHAPE = (6830702, 465)

In [6]:
def load_features():
    features = np.memmap(FEATURES_PATH, shape=FEATURES_SHAPE)
    target = np.load(TARGET_PATH)
    pdb_ids = np.load(GROUPS_PATH)
    
    return features, target, pdb_ids

In [7]:
features, target, pdb_ids = load_features()

In [8]:
def get_batch(index):
    with open(os.path.join(DATASET_DIR, 'batch_{}'.format(index)), 'rb') as infile:
        return dill.load(infile)
    

def get_data():
    return chain.from_iterable((get_batch(i) for i in tqdm(range(30))))

In [9]:
chain_ids = ['{}.cif1_{}'.format(sample.meta.pdb_id, sample.meta.pair.nt_left.chain_id) 
             for sample in get_data()]

100%|██████████| 30/30 [01:24<00:00,  2.83s/it]


In [10]:
with open('/home/mikhail/bioinformatics/data/nonredundant.txt', 'r') as infile:
    nonredundant_chain_ids = infile.read().splitlines()

In [11]:
nonredundant_mask = np.repeat([chain_id in nonredundant_chain_ids for chain_id in chain_ids], 2)

In [12]:
nonredundant_mask.sum()

1494852

nonredundant / all = 1494852 / 6807514 -- всего в 4 раза меньше, не так ужи плохо 

In [13]:
mask = (target != 0) & nonredundant_mask 

features = features[mask]
target = target[mask]
pdb_ids = pdb_ids[mask]

In [14]:
from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve

In [15]:
group_kfold = GroupKFold(n_splits=5)
group_kfold.get_n_splits(features, target, pdb_ids)

feature_importances = []
target_pred = np.zeros_like(target)
pred_proba = np.zeros_like(target, dtype=np.float32)

for train_index, test_index in group_kfold.split(features, target, pdb_ids):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = target[train_index], target[test_index]
    print('Train positive rate: {}, test positive rate: {}'.format(
        np.mean(y_train == 1), np.mean(y_test == 1)))
    
    model = RandomForestClassifier(class_weight='balanced', n_estimators=100, n_jobs=8)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    feature_importances.append(model.feature_importances_)
    target_pred[test_index] = y_pred
    pred_proba[test_index] = model.predict_proba(X_test)[:, 0]
    
    print(precision_recall_fscore_support(y_test, y_pred))

Train positive rate: 0.0028090995111181203, test positive rate: 0.002196188948895125
(array([0.84126984, 0.99788173]), array([0.03557047, 0.99998523]), array([0.06825499, 0.99893237]), array([  1490, 676958]))
Train positive rate: 0.0026193717645793075, test positive rate: 0.0023650688124221514
(array([0.7755102 , 0.99777971]), array([0.06138934, 0.99995787]), array([0.11377246, 0.9988676 ]), array([  1238, 522214]))
Train positive rate: 0.0024644285982737524, test positive rate: 0.003484320557491289
(array([1.        , 0.99669244]), array([0.0508982, 1.       ]), array([0.0968661 , 0.99834348]), array([  334, 95524]))
Train positive rate: 0.002454373188088759, test positive rate: 0.003630672926447574
(array([1.        , 0.99652528]), array([0.04310345, 1.        ]), array([0.08264463, 0.99825962]), array([  348, 95502]))
Train positive rate: 0.0024468860683922594, test positive rate: 0.0037310339109526574
(array([1.        , 0.99659949]), array([0.08888889, 1.        ]), array([0.1632